# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Jul 1 2022 11:25AM,240952,19,ADV80003750,"AdvaGen Pharma, Ltd",Released
1,Jul 1 2022 11:25AM,240952,19,ADV80003751,"AdvaGen Pharma, Ltd",Released
2,Jul 1 2022 11:08AM,240950,19,8217665,"Innogenix, LLC",Released
3,Jul 1 2022 11:08AM,240950,19,8217666,"Innogenix, LLC",Released
4,Jul 1 2022 11:08AM,240950,19,8217667,"Innogenix, LLC",Released
5,Jul 1 2022 11:08AM,240950,19,8217668,"Innogenix, LLC",Released
6,Jul 1 2022 11:08AM,240950,19,8217669,"Innogenix, LLC",Released
7,Jul 1 2022 10:26AM,240949,21,25732-COSN-1,"NBTY Global, Inc.",Released
8,Jul 1 2022 10:20AM,240948,12,U0954,"Uniderm USA, Inc",Executing
9,Jul 1 2022 10:10AM,240947,19,ACG-2102490508,ACG North America LLC,Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
24,240947,Released,2
25,240948,Executing,1
26,240949,Released,1
27,240950,Released,5
28,240952,Released,2


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Executing,Released
id,,
240947,NaN,2.0
240948,1.0,NaN
240949,NaN,1.0
240950,NaN,5.0
240952,NaN,2.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Executing,Released
id,,
240873,0.0,1.0
240878,0.0,1.0
240892,1.0,0.0
240893,0.0,1.0
240894,1.0,0.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus,Executing,Released
id,,
240873,0,1
240878,0,1
240892,1,0
240893,0,1
240894,1,0


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Executing,Released
0,240873,0,1
1,240878,0,1
2,240892,1,0
3,240893,0,1
4,240894,1,0


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus,id,Executing,Released
0,240873,,1
1,240878,,1
2,240892,1,
3,240893,,1
4,240894,1,


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Jul 1 2022 11:25AM,240952,19,"AdvaGen Pharma, Ltd"
2,Jul 1 2022 11:08AM,240950,19,"Innogenix, LLC"
7,Jul 1 2022 10:26AM,240949,21,"NBTY Global, Inc."
8,Jul 1 2022 10:20AM,240948,12,"Uniderm USA, Inc"
9,Jul 1 2022 10:10AM,240947,19,ACG North America LLC
11,Jul 1 2022 10:09AM,240946,10,"Nextsource Biotechnology, LLC"
13,Jul 1 2022 9:36AM,240940,12,Hush Hush
50,Jul 1 2022 9:27AM,240941,10,"Snap Medical Industries, LLC"
51,Jul 1 2022 9:17AM,240939,10,ISDIN Corporation
52,Jul 1 2022 9:14AM,240937,12,"NBTY Global, Inc."


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Executing,Released
0,Jul 1 2022 11:25AM,240952,19,"AdvaGen Pharma, Ltd",,2
1,Jul 1 2022 11:08AM,240950,19,"Innogenix, LLC",,5
2,Jul 1 2022 10:26AM,240949,21,"NBTY Global, Inc.",,1
3,Jul 1 2022 10:20AM,240948,12,"Uniderm USA, Inc",1,
4,Jul 1 2022 10:10AM,240947,19,ACG North America LLC,,2
5,Jul 1 2022 10:09AM,240946,10,"Nextsource Biotechnology, LLC",,2
6,Jul 1 2022 9:36AM,240940,12,Hush Hush,28,9
7,Jul 1 2022 9:27AM,240941,10,"Snap Medical Industries, LLC",,1
8,Jul 1 2022 9:17AM,240939,10,ISDIN Corporation,,1
9,Jul 1 2022 9:14AM,240937,12,"NBTY Global, Inc.",2,


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing
0,Jul 1 2022 11:25AM,240952,19,"AdvaGen Pharma, Ltd",2,
1,Jul 1 2022 11:08AM,240950,19,"Innogenix, LLC",5,
2,Jul 1 2022 10:26AM,240949,21,"NBTY Global, Inc.",1,
3,Jul 1 2022 10:20AM,240948,12,"Uniderm USA, Inc",,1
4,Jul 1 2022 10:10AM,240947,19,ACG North America LLC,2,
5,Jul 1 2022 10:09AM,240946,10,"Nextsource Biotechnology, LLC",2,
6,Jul 1 2022 9:36AM,240940,12,Hush Hush,9,28
7,Jul 1 2022 9:27AM,240941,10,"Snap Medical Industries, LLC",1,
8,Jul 1 2022 9:17AM,240939,10,ISDIN Corporation,1,
9,Jul 1 2022 9:14AM,240937,12,"NBTY Global, Inc.",,2


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing
0,Jul 1 2022 11:25AM,240952,19,"AdvaGen Pharma, Ltd",2,
1,Jul 1 2022 11:08AM,240950,19,"Innogenix, LLC",5,
2,Jul 1 2022 10:26AM,240949,21,"NBTY Global, Inc.",1,
3,Jul 1 2022 10:20AM,240948,12,"Uniderm USA, Inc",,1
4,Jul 1 2022 10:10AM,240947,19,ACG North America LLC,2,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

Column Completed does not exist yet


,Date,id,Warehouse,Customer,Released,Executing
0,Jul 1 2022 11:25AM,240952,19,"AdvaGen Pharma, Ltd",2.0,NaN
1,Jul 1 2022 11:08AM,240950,19,"Innogenix, LLC",5.0,NaN
2,Jul 1 2022 10:26AM,240949,21,"NBTY Global, Inc.",1.0,NaN
3,Jul 1 2022 10:20AM,240948,12,"Uniderm USA, Inc",NaN,1.0
4,Jul 1 2022 10:10AM,240947,19,ACG North America LLC,2.0,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing
0,Jul 1 2022 11:25AM,240952,19,"AdvaGen Pharma, Ltd",2.0,0.0
1,Jul 1 2022 11:08AM,240950,19,"Innogenix, LLC",5.0,0.0
2,Jul 1 2022 10:26AM,240949,21,"NBTY Global, Inc.",1.0,0.0
3,Jul 1 2022 10:20AM,240948,12,"Uniderm USA, Inc",0.0,1.0
4,Jul 1 2022 10:10AM,240947,19,ACG North America LLC,2.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing
Warehouse,,,
10,1927400,61.0,12.0
12,722825,9.0,31.0
15,481845,11.0,16.0
16,1204512,14.0,1.0
19,1204638,11.0,0.0
20,240894,0.0,1.0
21,240949,1.0,0.0
22,481821,0.0,2.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing
0,10,1927400,61.0,12.0
1,12,722825,9.0,31.0
2,15,481845,11.0,16.0
3,16,1204512,14.0,1.0
4,19,1204638,11.0,0.0
5,20,240894,0.0,1.0
6,21,240949,1.0,0.0
7,22,481821,0.0,2.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing
0,10,61.0,12.0
1,12,9.0,31.0
2,15,11.0,16.0
3,16,14.0,1.0
4,19,11.0,0.0
5,20,0.0,1.0
6,21,1.0,0.0
7,22,0.0,2.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,61.0
1,12,Released,9.0
2,15,Released,11.0
3,16,Released,14.0
4,19,Released,11.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,12,15,16,19,20,21,22
Status,,,,,,,,
Executing,12.0,31.0,16.0,1.0,0.0,1.0,0.0,2.0
Released,61.0,9.0,11.0,14.0,11.0,0.0,1.0,0.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,12,15,16,19,20,21,22
0,Executing,12.0,31.0,16.0,1.0,0.0,1.0,0.0,2.0
1,Released,61.0,9.0,11.0,14.0,11.0,0.0,1.0,0.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,12,15,16,19,20,21,22
0,Executing,12.0,31.0,16.0,1.0,0.0,1.0,0.0,2.0
1,Released,61.0,9.0,11.0,14.0,11.0,0.0,1.0,0.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()